# Test creating a binary mask

In [337]:
import geopandas as gpd
import leafmap
from shapely.ops import unary_union
from shapely.geometry import Point, mapping, box, shape
import shapely
import json
import os
import rasterio
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("..")

os.getcwd()
root = os.path.dirname(os.getcwd())
print(root)

from src.data.get_satellite_images import ReadSTAC

/Users/simonjasansky/programming/mine-segmentation


In [357]:
post_dataset = root + "/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg"
print(gpd.list_layers(post_dataset))
tiles = gpd.read_file(post_dataset, layer="tiles")
masks = gpd.read_file(post_dataset, layer="preferred_polygons")
masks

                 name geometry_type
0               tiles       Polygon
1       maus_polygons       Polygon
2       tang_polygons       Polygon
3         maus_bboxes       Polygon
4         tang_bboxes       Polygon
5  preferred_polygons       Polygon
6    preferred_bboxes       Polygon


,tile_id,geometry
0,2176,"POLYGON ((114.02234 58.18514, 114.02315 58.187..."
1,1866,"POLYGON ((95.60614 24.66162, 95.61346 24.67116..."
2,653,"POLYGON ((-61.42576 4.8384, -61.42598 4.83972,..."
3,2749,"POLYGON ((132.80162 52.94666, 132.80151 52.946..."
4,152,"POLYGON ((-111.3945 56.9912, -111.39321 56.990..."
5,288,"POLYGON ((-92.5765 47.364, -92.5699 47.3663, -..."
6,2779,"POLYGON ((137.14321 -32.72606, 137.14287 -32.7..."
7,577,"POLYGON ((119.4193 -27.28, 119.4188 -27.2783, ..."
8,2503,"POLYGON ((-56.73965 -7.67412, -56.74183 -7.672..."
9,803,"POLYGON ((114.56123 -0.67777, 114.56159 -0.678..."


In [340]:
# select first row
row = tiles.iloc[0]
row

bounds = row.geometry.bounds
bounds


# get lat and lon
lat = row.geometry.centroid.y
lon = row.geometry.centroid.x
lat, lon

(47.41662619753377, -92.58333336689567)

In [341]:
# Download the image 
api_url="https://planetarycomputer.microsoft.com/api/stac/v1"
bands = ['B04', 'B03', 'B02']

stac_reader = ReadSTAC(api_url=api_url, collection = "sentinel-2-l2a")

item = stac_reader.get_item_by_name(row.s2_tile_id, bbox=bounds)

# read as stack
stack = stac_reader.get_stack(
    items=item, 
    bands=bands,
    crop_to_bounds=False, 
    squeeze_time_dim=True,
    custom_point_and_buffer=[lon, lat, 10240],
    chunk_size=512,
    )

# persist to disk
# stac_reader.save_stack_as_geotiff(stack, root + f"/data/processed/images/{row.tile_id}_{row.s2_tile_id}_img.tif")

Loading stack...
Returning stack from single S2 image with ID: S2B_MSIL2A_20190731T165859_R069_T15TWN_20201005T203233


In [ ]:
# show the stack 
stac_reader.display_stack_as_image(stack)

In [343]:
# read the mask 
tile_id = row.tile_id
poly = masks[masks.tile_id == tile_id].geometry.values

# convert to same crs as stack
poly = poly.to_crs(stack.crs)

In [348]:
mask_raster = rasterio.features.rasterize(poly, out_shape=(2048,2048), transform=stack.rio.transform())

np.unique(mask_raster)

# show the image
plt.imshow(mask_raster)

array([0, 1], dtype=uint8)

# Testing (Delete)

In [304]:
import rasterio
from rasterio.features import rasterize
from shapely.geometry import Polygon

# Define your Shapely polygon (example polygon in EPSG:32620)
polygon = Polygon([(500000, 500000), (500000, 510000), (510000, 510000), (510000, 500000)])

# Define the spatial resolution and bounds of your output raster
transform = rasterio.transform.from_origin(west=500000, north=510000, xsize=10, ysize=10)
width, height = 2048, 2048  # dimensions of the raster

# Define the output raster's CRS
crs = "EPSG:32620"

# Rasterize the polygon
binary_mask = rasterize(
    [(polygon, 1)],
    out_shape=(height, width),
    transform=transform,
    fill=0,  # background value
    all_touched=True,
    dtype='uint8'
)

# binary_mask now contains the binary mask of the rasterized polygon
np.unique(binary_mask)

plt.imshow(binary_mask)

array([0, 1], dtype=uint8)